# Sentimet Analysis

Demo tutorial for how to use nlp_toolkit to train classification model and predict new samples. The task we choose is sentiment binary classification.

The dataset is crawled from Kanzhun.com and Dajie.com, which is about company pros and cons.

Available models:
    1. Bi-LSTM Attention
    2. Multi Head Self Attention
    3. TextCNN
    4. DPCNN

In [1]:
import sys
sys.path.append('../')
from nlp_toolkit import Dataset, Classifier

D:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Data Processing

### Load config dict

In [2]:
import yaml
config = yaml.load(open('../config_classification.yaml', encoding='utf8'))
#config['data']['basic_token'] = 'char'
#config['embed']['pre'] = False

### Load Data

In [3]:
dataset = Dataset(fname='../data/company_pro_con.txt', task_type='classification', mode='train', config=config)

2018-11-24 23:27:51,934 - data.py[line:136] - INFO: data loaded


In [4]:
for x, y in zip(dataset.texts[0:10], dataset.labels[0:10]):
    print(x, y)

进去 前 许诺 的 工资 给 的 高 0
校园 环境 优美 ， 美女 很多 ， 适合 居住 ， 食堂 饭菜 便宜 ， 操场 好 ， 可以 天天 运动 0
老板 人 很好 老 员工 会 各种 教 你 东西 ， 而且 不会 有所 保留 薪水 在 大连 还 算 可以 0
人员 比较 多 ， 复杂   办公室 容易 形成 拉帮结派 不利于 企业 发展 1
出差 太多 了 。 在 现场 开发 很苦 逼 。 1
公司 目前 地理位置 不 太 理想 ， 离 城市 中心 较 远点 。 1
公司 的 技术 水平 国内 顶尖 ， 十几 年 的 资历 ， 制作 的 作品 几乎 都 是 精品 ， 参与 过 很多 知名 项目 。 0
工作 流程 复杂     个人 上升 空间 有限     新产品 的 创新 能力 有限   组织 架构 稍 显 臃肿 1
无偿 加班 ， 加班 多 ， 没 加班费 ， 压力 很大 1
环境 比较 轻松 ， 跟 项目 走 ， 能 学 不少 专业 知识 ， 经验 很 重要 0


### Transform data to index

In [5]:
# if we want to use pre_trained embeddings, we need a gensim-format embedding file
x, y, config = dataset.transform()
print(x['token'].shape, y.shape)

2018-11-24 23:28:00,514 - data.py[line:214] - INFO: texts and labels have been transformed to number index
2018-11-24 23:28:05,272 - utilities.py[line:226] - INFO: OOV rate: 0.00 %
2018-11-24 23:28:05,288 - data.py[line:220] - INFO: Loaded Pre_trained Embeddings


(94635, 100) (94635, 2)


if your want to see the vocab and label index mapping dict

In [6]:
# dataset.transformer._word_vocab._token2id

In [7]:
# dataset.transformer._label_vocab._token2id

In [6]:
transformer = dataset.transformer

## Classifier Training

### Define classifier

avialiable models: model_name_list = ['dpcnn', 'text_cnn', 'bi_lstm_att', 'multi_head_self_att']

In [7]:
model_name='bi_lstm_att'
# if you want to get attention weights during prediction, please set return_attention=True, only for bi_lstm_att model
config['model'][model_name]['return_att'] = True
# text_cnn and dpcnn currently not support bucket sequence type
text_classifier = Classifier(model_name=model_name, transformer=transformer, seq_type='bucket', config=config)

### Train Model

In [8]:
trained_model = text_classifier.train(x, y)

2018-11-24 23:28:21,606 - trainer.py[line:129] - INFO: bi_lstm_att model structure...
2018-11-24 23:28:21,632 - trainer.py[line:139] - INFO: train/valid set: 75708/18927
2018-11-24 23:28:21,633 - trainer.py[line:89] - INFO: use bucket sequence to speed up model training
2018-11-24 23:28:21,635 - sequence.py[line:346] - INFO: Training with 99 non-empty buckets


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
token (InputLayer)              (None, None)         0                                            
__________________________________________________________________________________________________
token_embeddings (Embedding)    (None, None, 300)    7333200     token[0][0]                      
__________________________________________________________________________________________________
spatial_dropout1d_1 (SpatialDro (None, None, 300)    0           token_embeddings[0][0]           
__________________________________________________________________________________________________
activation_1 (Activation)       (None, None, 300)    0           spatial_dropout1d_1[0][0]        
__________________________________________________________________________________________________
bi_lstm_0 

2018-11-24 23:28:21,895 - sequence.py[line:346] - INFO: Training with 99 non-empty buckets


mointor training process using f1 score
Successfully made a directory: models\bi_lstm_att_201811242328
using Early Stopping
using Reduce LR On Plateau
tracking loss history and metrics


2018-11-24 23:28:22,211 - trainer.py[line:171] - INFO: saving model parameters and transformer to models\bi_lstm_att_201811242328


model hyperparameters:
 {'nb_classes': 2, 'nb_tokens': 24444, 'maxlen': None, 'embedding_dim': 300, 'rnn_size': 512, 'attention_dim': 128, 'embed_dropout_rate': 0.15, 'final_dropout_rate': 0.5, 'return_attention': True}
Epoch 1/25
1230/1230 [==============================] - 125s 102ms/step - loss: 0.4283 - acc: 0.7401 - val_loss: 0.2717 - val_acc: 0.8130
0 - f1: 92.99
1 - f1: 91.98

Epoch 00001: f1 improved from -inf to 0.92509, saving model to models\bi_lstm_att_201811242328\model_weights_01_0.8130_0.9251.h5
Epoch 2/25
1230/1230 [==============================] - 120s 97ms/step - loss: 0.2227 - acc: 0.8430 - val_loss: 0.2179 - val_acc: 0.8461
0 - f1: 94.42
1 - f1: 93.85

Epoch 00002: f1 improved from 0.92509 to 0.94149, saving model to models\bi_lstm_att_201811242328\model_weights_02_0.8461_0.9415.h5
Epoch 3/25
1230/1230 [==============================] - 120s 97ms/step - loss: 0.1648 - acc: 0.8721 - val_loss: 0.2083 - val_acc: 0.8514
0 - f1: 94.55
1 - f1: 94.04

Epoch 00003: f1 impr

### 10-fold training

In [10]:
config['train']['train_mode'] = 'fold'
text_classifier_new = Classifier(model_name=model_name, transformer=transformer, seq_type='bucket', config=config)
text_classifier_new.train(x, y)

2018-11-25 10:22:05,602 - trainer.py[line:186] - INFO: 10-fold starts!



------------------------ fold 0------------------------


2018-11-25 10:22:08,931 - trainer.py[line:199] - INFO: bi_lstm_att model structure...
2018-11-25 10:22:08,948 - trainer.py[line:89] - INFO: use bucket sequence to speed up model training
2018-11-25 10:22:08,950 - sequence.py[line:346] - INFO: Training with 99 non-empty buckets


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
token (InputLayer)              (None, None)         0                                            
__________________________________________________________________________________________________
token_embeddings (Embedding)    (None, None, 300)    7333200     token[0][0]                      
__________________________________________________________________________________________________
spatial_dropout1d_3 (SpatialDro (None, None, 300)    0           token_embeddings[0][0]           
__________________________________________________________________________________________________
activation_3 (Activation)       (None, None, 300)    0           spatial_dropout1d_3[0][0]        
__________________________________________________________________________________________________
bi_lstm_0 

2018-11-25 10:22:09,246 - sequence.py[line:346] - INFO: Training with 96 non-empty buckets


mointor training process using f1 score
using Early Stopping
using Reduce LR On Plateau
tracking loss history and metrics
Epoch 1/25
1384/1384 [==============================] - 135s 98ms/step - loss: 0.9319 - acc: 0.8271 - val_loss: 0.8424 - val_acc: 0.8466
0 - f1: 87.52
1 - f1: 86.96
Epoch 2/25
 330/1384 [======>.......................] - ETA: 1:34 - loss: 0.8630 - acc: 0.8386

KeyboardInterrupt: 

## Predict New Samples

## Load data and transformer

In [12]:
dataset = Dataset(mode='predict', fname='../data/company_pro_con_predict.txt',
                  task_type='classification',
                  tran_fname='models/bi_lstm_att_201811242328/transformer.h5')
x_seq = dataset.transform()

2018-11-25 10:26:01,727 - data.py[line:98] - INFO: transformer loaded
2018-11-25 10:26:01,916 - data.py[line:136] - INFO: data loaded


## Load model

In [14]:
text_classifier = Classifier('bi_lstm_att', dataset.transformer)
text_classifier.load(weight_fname='models/bi_lstm_att_201811242328/model_weights_07_0.8364_0.9436.h5',
                     para_fname='models/bi_lstm_att_201811242328/model_parameters.json')

model loaded


## predict samples

In [15]:
y_pred, attention = text_classifier.predict(x_seq)

2018-11-25 10:32:11,782 - classifier.py[line:144] - INFO: predict 94635 samples used 169.0s


### attention visualization

currently only support for bi_lstm_att model

In [16]:
x_len = x_seq['length']
attention_true = [attention[i][:x_len[i]] for i in range(len(x_len))]

In [17]:
from nlp_toolkit import visualization as vs

In [19]:
vs.att2html(dataset.texts[2], attention_true[2])

' <span style="background-color: #FFFEFE">老板</span> <span style="background-color: #FFFEFE">人</span> <span style="background-color: #FFB9B9">很好</span> <span style="background-color: #FFFCFC">老</span> <span style="background-color: #FFFEFE">员工</span> <span style="background-color: #FFFEFE">会</span> <span style="background-color: #FFF9F9">各种</span> <span style="background-color: #FFB5B5">教</span> <span style="background-color: #FFCBCB">你</span> <span style="background-color: #FFFEFE">东西</span> <span style="background-color: #FFFEFE">，</span> <span style="background-color: #FFFEFE">而且</span> <span style="background-color: #FFEFEF">不会</span> <span style="background-color: #FFFEFE">有所</span> <span style="background-color: #FFFEFE">保留</span> <span style="background-color: #FFFEFE">薪水</span> <span style="background-color: #FFFEFE">在</span> <span style="background-color: #FFFEFE">大连</span> <span style="background-color: #FFFEFE">还</span> <span style="background-color: #FFFCFC">算</span> <span s

<span style="background-color: #FFFEFE">老板</span> <span style="background-color: #FFFEFE">人</span> <span style="background-color: #FFD6D6">很好</span> <span style="background-color: #FFFCFC">老</span> <span style="background-color: #FFFEFE">员工</span> <span style="background-color: #FFFEFE">会</span> <span style="background-color: #FFFBFB">各种</span> <span style="background-color: #FFD3D3">教</span> <span style="background-color: #FFD7D7">你</span> <span style="background-color: #FFECEC">东西</span> <span style="background-color: #FFFCFC">，</span> <span style="background-color: #FFFEFE">而且</span> <span style="background-color: #FFD4D4">不会</span> <span style="background-color: #FFFEFE">有所</span> <span style="background-color: #FFFEFE">保留</span> <span style="background-color: #FFFEFE">薪水</span> <span style="background-color: #FFFEFE">在</span> <span style="background-color: #FFFEFE">大连</span> <span style="background-color: #FFFEFE">还</span> <span style="background-color: #FFEDED">算</span> <span style="background-color: #FFD5D5">可以</span><br><br>

or you can write all results to html file and open it in a browser

In [ ]:
vs.attention_visualization(dataset.texts, attention_true, x_len, output_fname='result.html')